##### Tzanis Anevlavis and Paulo Tabuada, "A simple hierarchy for computing controlled invariant sets", Submitted to 2020 ACM 23rd International Conference on Hybrid Systems: Computation and Control (HSCC'2020).
##### Section 5: Example 2:
In this notebook, we compute ellipsoidal controlled invariant sets using the method proposed in [LB18].

Then we compare its results with our proposed method for `L=2` to `L=6`, and the one in [AT19].

[AT19] Tzanis Anevlavis and Paulo Tabuada, "Computing controlled invariant sets in two moves",  In 2019 IEEE Conference on Decision and Control (CDC). 6249–6254.

[LB18] B. Legat, P. Tabuada, R. M. Jungers. Computing controlled invariant sets for hybrid systems with applications to model-predictive control, IFAC-PapersOnLine, Volume 51, Issue 16, 2018, Pages 193-198, ISSN 2405-8963.

The original code of [LB18] can be found in the following [github repository](https://github.com/blegat/SwitchOnSafety.jl).

We begin by formulating the problem.

In [1]:
# Load dependencies:
using MosekTools
using JuMP
using Polyhedra
using MathematicalSystems
using SwitchOnSafety

# Choose solver:
solver = with_optimizer(Mosek.Optimizer, QUIET=true)

# Safe Set:
#values taken from the corresponding MATLAB code in './paper-examples/example_1_2.m' .
G =[ 0.9147 -0.5402
     0.2005  0.6213
    -0.8193  0.9769
    -0.4895 -0.8200
     0.7171 -0.3581
     0.8221  0.0228
     0.3993 -0.8788]
F = [0.5566
     0.8300
     0.7890
     0.3178
     0.4522
     0.7522
     0.1099]
safe_set = polyhedron(hrep(G, F), DefaultLibrary{Float64}(solver))
# Input Constraints:
input_set = polyhedron(convexhull([-1], [1]))

# Define system:
A = [1.5 1.0
     0.0 1.0]
B = reshape([0.5, 0.25], 2, 1)

system = ConstrainedLinearControlDiscreteSystem(A, B, safe_set, input_set)

┌ Info: Recompiling stale cache file /Users/j10/.julia/compiled/v1.0/MosekTools/UJwlm.ji for MosekTools [1ec41992-ff65-5c91-ac43-2df89e9693a4]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/j10/.julia/compiled/v1.0/JuMP/DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/j10/.julia/compiled/v1.0/Polyhedra/17i4E.ji for Polyhedra [67491407-f73d-577b-9b50-8179a7c68029]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/j10/.julia/compiled/v1.0/MathematicalSystems/6oLdk.ji for MathematicalSystems [d14a8603-c872-5ed3-9ece-53e0e82e39da]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/j10/.julia/compiled/v1.0/SwitchOnSafety/EUPLd.ji for SwitchOnSafety [ceb7f16a-07bf-5f4a-9354-b68f01b1610f]
└ @ Base loading.jl:1190


ConstrainedLinearControlDiscreteSystem{Float64,Array{Float64,2},Array{Float64,2},DefaultPolyhedron{Float64,MixedMatHRep{Float64,Array{Float64,2}},MixedMatVRep{Float64,Array{Float64,2}}},Interval{Int64,StaticArrays.SArray{Tuple{1},Int64,1,1},StaticArrays.Size{(1,)}}}([1.5 1.0; 0.0 1.0], [0.5; 0.25], HalfSpace([0.9147, -0.5402], 0.5566) ∩ HalfSpace([0.2005, 0.6213], 0.83) ∩ HalfSpace([-0.8193, 0.9769], 0.789) ∩ HalfSpace([-0.4895, -0.82], 0.3178) ∩ HalfSpace([0.7171, -0.3581], 0.4522) ∩ HalfSpace([0.8221, 0.0228], 0.7522) ∩ HalfSpace([0.3993, -0.8788], 0.1099), HalfSpace([1], 1) ∩ HalfSpace([-1], 1) : convexhull([1], [-1]))

Compute the invariant set by searching for any ellipsoid with a given point in its interior.
As the system is reformulated into an algebraic system with safe set `safe_set * input_set`, the Chebyshev center is `(cheby_center, 0)`.

To avoid having to solve Bilinear Matrix Inequalities, we set the S-procedure scaling to `1.18` (found by a few trials, checking what gives the best `objective_value`).

In [2]:
# Compute controlled invariant maximum volume ellipsoid inside the safe set:
# Choose:
# S-procedure constant:
S_procedure_scaling = 1.18
# Center of ellipsoid:
cheby_center, cheby_radius = chebyshevcenter(safe_set, solver)
# cntr = [cheby_center; 0.0]
cntr = [0.0;0.0;0.0]

using SwitchOnSafety
variable = Ellipsoid(point = SetProg.InteriorPoint(cntr))
max_vol_ell = invariant_set(system, solver, variable, λ = S_procedure_scaling)

MOI.get(model, MOI.SolveTime()) = 0.0833899974822998
JuMP.termination_status(model) = OPTIMAL::TerminationStatusCode = 1
JuMP.primal_status(model) = FEASIBLE_POINT::ResultStatusCode = 1
JuMP.dual_status(model) = FEASIBLE_POINT::ResultStatusCode = 1
JuMP.objective_value(model) = 0.06568238083830727


SetProg.Sets.Translation{SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}},Float64,Array{Float64,1}}(SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}}(SetProg.Sets.EllipsoidAtOrigin{Float64}([0.0900401 0.0158536; 0.0158536 0.0594409])), [-0.0897293, 0.0864089])

Comparison plot: we import the results for [AT19] and our CIS hierarchy from MATLAB, computed in `./paper-examples/example_1_2.m`.

Corresponds to Figure 5, in the paper "A simple hieararchy for computing controlled invariant sets".

In [11]:
# MCIS of safe_set for system
# MCIS = {x\in \R^2 | Gex x <= Fex}:
G0 = [  0.637084683951207   0.770793815150842
        0.775234985517289   0.631672951162236
        0.976187060183953   0.216930457818656
        0.832050294337844  -0.554700196225229
       -0.991877357182905   0.127197909997987
       -0.819300000000000   0.976900000000000
       -0.489500000000000  -0.820000000000000
        0.399300000000000  -0.878800000000000]
F0 = [  0.552086265170460
        0.462171503398584
        0.330633376692741
        0.145371532267275
        0.770310058691805
        0.789000000000000
        0.317800000000000
        0.109900000000000]

# CIS from the algorithm in [AT19]
# CISL1 = {x\in \R^2 | G1 x <= F1}:
G1 = [ -0.447213595499958  -0.894427190999916
        0.447213595499958   0.894427190999916
       -0.512569589886253  -0.858645686837032
        0.413670248214957  -0.910426782197105
        0.832050294337844  -0.554700196225229
       -0.642595923761930   0.766205245847711
       -0.832050294337844   0.554700196225229]
F1 = [  0.298142396999972
        0.298142396999972
        0.332777560093669
        0.113855147204668
        0.145371531096108
        0.618830933538585
        0.669330460442295]

# CIS from our hierarchy
# L=2
# CISL2 = {x\in \R^2 | G2 x <= F2}:
G2 = [  0.447213595499958   0.894427190999916
       -0.990551210330747   0.137143354604923
        0.832050294337844  -0.554700196225229
        0.963676737832102   0.267071422958874
       -0.512569589886253  -0.858645686837032
        0.413670248214957  -0.910426782197105
       -0.642595923761930   0.766205245847711]
F2 = [  0.417399355799961
        0.769802748026351
        0.145371531096107
        0.336159110617651
        0.332777560093669
        0.113855147204668
        0.618830933538585]

# CIS from our hierarchy
# L=3
# CISL3 = {x\in \R^2 | G3 x <= F3}:
G3 = [  0.447213595499958   0.894427190999916
        0.975240941814010   0.221144987304081
        0.832050294337843  -0.554700196225229
       -0.991877357182905   0.127197909997987
        0.954642451034589   0.297754581295858
        0.413670248214957  -0.910426782197105
       -0.512569589886253  -0.858645686837032
       -0.642595923761930   0.766205245847711]
F3 = [  0.455059448052589
        0.331129208675194
        0.145371531096107
        0.770310058691805
        0.351971838818794
        0.113855147204668
        0.332777560093669
        0.618830933538585]

# CIS from our hierarchy
# L=4
# CISL4 = {x\in \R^2 | G4 x <= F4}:
G4 = [ -0.991877357182905   0.127197909997987
        0.974603867419725   0.223935931932582
        0.747168092140333   0.664635119510980
        0.832050294337844  -0.554700196225229
        0.976105517633115   0.217297074177701
        0.433393544393727   0.901204769005270
       -0.642595923761930   0.766205245847711
       -0.512569589886253  -0.858645686837032
        0.413670248214957  -0.910426782197105]
F4 = [  0.770310058691805
        0.332558022926511
        0.465288894726633
        0.145371531096108
        0.330676724847788
        0.524280662494074
        0.618830933538585
        0.332777560093669
        0.113855147204668]

# CIS from our hierarchy
# L=5
# CISL5 = {x\in \R^2 | G5 x <= F5}:
G5 = [ -0.991877357182905   0.127197909997987
        0.976152664670240   0.217085179727198
        0.832050294337844  -0.554700196225229
        0.975525606181555   0.219885860582507
        0.763489515330491   0.645820222647458
        0.743560611919875   0.668668540011672
        0.447213595499958   0.894427190999916
       -0.642595923761930   0.766205245847711
        0.413670248214957  -0.910426782197105
       -0.512569589886253  -0.858645686837032]
F5 = [  0.770310058691805
        0.331349289307086
        0.145449366486622
        0.332145065120396
        0.467814656600340
        0.474198877999993
        0.566611854014165
        0.618830933538585
        0.113855147204668
        0.332777560093669]

# using MAT
# vars = matread("mat2julia.mat")

mcis = polyhedron(hrep(G0,F0), DefaultLibrary{Float64}(solver))
cis1 = polyhedron(hrep(G1,F1), DefaultLibrary{Float64}(solver))
cis2 = polyhedron(hrep(G2,F2), DefaultLibrary{Float64}(solver))
cis3 = polyhedron(hrep(G3,F3), DefaultLibrary{Float64}(solver))
cis4 = polyhedron(hrep(G4,F4), DefaultLibrary{Float64}(solver))
cis5 = polyhedron(hrep(G5,F5), DefaultLibrary{Float64}(solver))

using Plots
plot(safe_set, color=:blue,fmt = :svg, dpi=300)
plot!(mcis, color=:lightgrey)
plot!(cis5, color=:darkgreen)
plot!(cis4, color=:green)
plot!(cis3, color=:lime)
plot!(cis2, color=:lightgreen)
plot!(cis1, color=:white)
plot!(project(max_vol_ell, 1:2), color=:orange)
xaxis!([-0.8,-0.6,-0.4,-0.2,0,0.2,0.4,0.6,0.8,1.0])
yaxis!([-0.2,0,0.2,0.4,0.6,0.8,1.0,1.2])

savefig("figure5_2")